<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / Assignment 4*

---
# N414. 신경망과 학습에 관련된 파라미터 튜닝 (HyperTune)



## 실전 연습과제

다음 통신사 고객 이탈(Churn) 데이터셋에서 정확도를 조정해보는 파라미터 학습을 진행해보겠습니다 : [다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/telecom/TelcomCustomer.csv)

## 진행방식

- 데이터를 다운로드 받고 읽어옴(load)
- 데이터 클리닝을 진행 (필수는 아니지만 추천)
- Keras MLP model을 만들고, 학습 진행
- Hyperparameter 튜닝 진행:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (optimizer에 따라서 해당되면)
 - momentum (optimizer에 따라서 해당되면)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
하이퍼 파라미터의 초기 패스를 위해 그리드 검색 및 교차 검증을 사용할 수 있어야 합니다. 

실제 큰 통신회사의 데이터이기 때문에 최대한 정확하게 파악해 보십시오! 


# 데이터 전처리 

### 문항 1) 내 로컬 파일을 colab에 업로드하기

충분한 GPU를 가지고 있다면, 쉽게 문제를 해결할 수 있겠지만, 제한된 자원에서 충분한 GPU를 제공받지 못할 지 모릅니다. 이럴 때에는 딥러닝 커뮤니티에 물어볼 수도 있겠지만, 가성비 좋은 Colab의 GPU를 이용해서 실제 GPU사용량을 예측할 수 있다면 좋겠습니다. 모델 파라미터의 개수와 batch size 등이 GPU메모리에 큰 영향을 미치니 여러가지로 활용해보시기 바랍니다.

Colab의 GPU를 이용하기 위해서, 로컬로 진행하시던 분들도 이번에는 colab을 사용해봅시다. 

- 구글에서 colab의 라이브러리를 찾아서 업로드하세요. 
이후에 Pandas를 이용하여 데이터프레임으로 저장합니다.

colab을 사용하기위해서는 colab 라이브러리들을 잘 활용할 수 있으면 좋습니다. colab 기본형의 GPU의 사용제한 때문에 하지 못하는 일이 있다면, colab pro를 사용할 수 있습니다. 코드스테이츠에서 제공하는 colab pro 설치 가이드가 있지만, 스스로 문제를 해결해보시면 좋습니다. 그러나 도움이 필요하시면 HelpDesk에 문의해주시기 바랍니다.

### 로컬 파일을 업로드하는 코드를 입력하세요. 

In [ ]:
##### Your Code Here #####
# 로컬 파일 업로드

# 데이터 불러오기
import pandas as pd
df = __________('TelcomCustomer.csv')

Saving TelcomCustomer.csv to TelcomCustomer.csv


In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### 문항 2) 결측치가 있는지 isnull()함수를 이용하여 확인하고 결과값을 입력하시오. 

In [ ]:
##### Your Code Here #####
# 결측치 확인


0

### 문항 3) dtypes를 이용해서 데이터 타입을 확인하고 아래 문제에 답하시오.

TotalCharges와 같이 중요한 타켓값이 숫자로 되어있어야 하는데, object로 되어있는 것을 확인하고 숫자형으로 바꿔주세요. <br>
숫자형이 아닌 결측치의 개수는 몇개인가요? 

In [ ]:
##### Your Code Here #####
# 데이터 타입 확인

# TotalCharges 숫자형으로 변환

# 결측치 확인

# 결측치 드랍


In [ ]:
# customerID 드랍
df = df.drop(columns='customerID')

In [ ]:
# Attributing No internet service to No
no_internet_feats = [ 'TechSupport','OnlineBackup', 'DeviceProtection','StreamingTV',
                 'OnlineSecurity','StreamingMovies']

for i in no_internet_feats:
    df[i] = df[i].replace({'No internet service':'No'})

# Attributing No phone service to No
df['MultipleLines']=df['MultipleLines'].replace({'No phone service':'No'})

# Attributing No phone service to No
df['SeniorCitizen']=df['SeniorCitizen'].replace({0:'No',
                                                 1:'Yes'})

In [ ]:
!pip install category_encoders

In [ ]:
df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [ ]:
# 원핫 인코딩
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df)

In [ ]:
df_encoded.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,1
1,2,1,2,1,34,2,1,1,2,2,2,1,1,1,2,2,2,56.95,1889.50,1
2,2,1,2,1,2,2,1,1,2,1,1,1,1,1,1,1,2,53.85,108.15,2
3,2,1,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75,1
4,1,1,2,1,2,2,1,2,1,2,1,1,1,1,1,1,1,70.70,151.65,2


In [ ]:
# 타겟을 0과 1로 바꿔주기
df_encoded['Churn'] = df_encoded['Churn'].replace({1: 0, 2: 1})
df_encoded['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [ ]:
df_encoded.shape

(7032, 20)

### 문항 4) 훈련집합과 테스트 집합을 나누는 코드를 만들고, 해당 코드를 입력하시오.

- random_state=1
- test_size=0.25

In [ ]:
##### Your Code Here #####
# 훈련, 테스트 셋을 나누기
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_encoded, stratify=df_encoded['Churn'], random_state=1)
train.shape, test.shape

((5274, 20), (1758, 20))

In [ ]:
# features 와 target 을 분리

target = 'Churn'
features = df_encoded.drop(columns=[target]).columns

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

### 문항 5) sklearn.preprocessing.StandardScaler를 이용하여 정규화를 진행하시고, 문제를 보고 빈칸에 알맞은 단어를 넣으시오.

`X_train_scaled = scaler.##### Your Code Here #####`

In [ ]:
##### Your Code Here #####
# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.##### Your Code Here #####(X_train)
X_test_scaled = scaler.##### Your Code Here #####(X_test)

In [ ]:
# 19x1 형태 => 입력층의 노드수가 19개! 
X_train_scaled[0]

array([ 0.99508225, -0.43925252,  0.96350214, -0.65400354, -0.09779796,
        0.32541085,  1.16743446, -1.19019364,  1.59286639,  0.72650531,
       -0.72042294, -0.64249881, -0.79264136,  1.25707392, -0.83276144,
       -0.83448511, -0.27868378, -0.00881575, -0.17257846])

# 모델링

## 기본 모델

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
import tensorflow as tf
import IPython
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=20c4dca344cfa42b39d5676c3c929d454fb4bc306218f9e3a6e2de4e5da71bce
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=37a3265106d30c399116ef2ee7bada77afad24d874f28d74a3289efd27b15139
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


### 문제 6. np.unique를 이용해서 y_train의 Class 의 개수를 확인하고 입력하시오.

In [ ]:
##### Your Code Here #####

# 이진분류의 경우 class는 2개이지만, 출력층의 노드수 = 1 

In [ ]:
# 간단한 모델 만들어서 성능을 보기 !
tf.random.set_seed(7)

model2 = Sequential()
model2.add(Dense(64, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid')) # 00분류니까 노드수 1, 활성함수로는 시그모이드

model2.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model2.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled,y_test))

Epoch 1/10
165/165 [==============================] - 1s 4ms/step - loss: 0.5154 - accuracy: 0.7276 - val_loss: 0.4502 - val_accuracy: 0.7912
Epoch 2/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4173 - accuracy: 0.8054 - val_loss: 0.4468 - val_accuracy: 0.7986
Epoch 3/10
165/165 [==============================] - 0s 2ms/step - loss: 0.4056 - accuracy: 0.8073 - val_loss: 0.4493 - val_accuracy: 0.7912
Epoch 4/10
165/165 [==============================] - 0s 2ms/step - loss: 0.3986 - accuracy: 0.8095 - val_loss: 0.4460 - val_accuracy: 0.8020
Epoch 5/10
165/165 [==============================] - 0s 2ms/step - loss: 0.3985 - accuracy: 0.8119 - val_loss: 0.4516 - val_accuracy: 0.7935
Epoch 6/10
165/165 [==============================] - 0s 2ms/step - loss: 0.3901 - accuracy: 0.8207 - val_loss: 0.4535 - val_accuracy: 0.7907
Epoch 7/10
165/165 [==============================] - 0s 2ms/step - loss: 0.3903 - accuracy: 0.8156 - val_loss: 0.4530 - val_accuracy: 0.7969
Epoch 

In [ ]:
# 테스트셋 사용해서 결과 보기
model2.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4643 - accuracy: 0.7838


[0.4642634391784668, 0.7838453054428101]

파라미터 튜닝을 하기전에 간단히 임의로 넣어본 결과도 꽤 좋다. 이젠 GridSearchCV 를 사용해서 튜닝을 해보겠음!



## GridSearchCV 사용

In [ ]:
# 모델 만들기
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn=model_builder, verbose=0)

# GridSearch
batch_size = [50, 100, 300]
epochs = [10, 20, 30]
nodes = [64, 128, 256]
activation = ['relu', 'sigmoid']
param_grid = dict(batch_size=batch_size, epochs=epochs, nodes=nodes, activation=activation)


# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  4.6min finished


In [ ]:
# 최적의 결과값을 낸 파라미터를 출력합니다
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.806219200293223 using {'activation': 'sigmoid', 'batch_size': 100, 'epochs': 30, 'nodes': 128}
Means: 0.7971179485321045, Stdev: 0.003950068993039538 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 64}
Means: 0.7929465174674988, Stdev: 0.003251132129816269 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 128}
Means: 0.7857413490613302, Stdev: 0.0032950603139239336 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 256}
Means: 0.7899127801259359, Stdev: 0.0007094483673364635 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 64}
Means: 0.7789154450098673, Stdev: 0.010685647465184307 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 128}
Means: 0.7715206742286682, Stdev: 0.004510958934156072 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 256}
Means: 0.7851725618044535, Stdev: 0.009745994559001799 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 30, '

### 문항 7) 최적의 결과를 낸 파라미터와 결과값을 입력하시오.

- batch_size
- epochs
- nodes
- activation

- 정답은 `[100, 100, 100], activation_name` 형태로 입력하시오.

'activation': 'sigmoid', 'batch_size': 100, 'epochs': 30, 'nodes': 128 가 최적으로 나왔다. 정확도는 약 0.80 

이젠 Keras Tuner 를 사용한 파라미터 튜닝도 해보겠음!

## Keras Tuner 사용

In [ ]:
# 모델 만들기

def model_builder(hp):

  model = Sequential()

  # Dense layer에서 노드 수를 조정(32-512)
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  model.add(Dense(units = hp_units, activation='relu'))
  model.add(Dense(units = hp_units, activation='relu'))

  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  # 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
  model.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss=keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics=['accuracy'])

  return model

In [ ]:
# 튜너를 인스턴스화하고 하이퍼 튜닝을 수행

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 30, 
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [ ]:
# callback 정의 : 하이퍼 파라미터 검색을 실행하기 전에 모든 교육 단계가 끝날 때마다 교육 출력을 지우도록 콜백을 정의합니다.

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs = 30, batch_size=50, validation_data = (X_test_scaled,y_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
최적화된 Dense 노드 수 : {best_hps.get('units')} 
최적화된 Learning Rate : {best_hps.get('learning_rate')} 
""")

### 문항 8) Keras 튜너를 활용하여 얻어낸 파라미터를 입력하시오. 

- 정답은 `[100, 100]` 형태로 입력하시오.

In [ ]:
# 찾은 파라미터들로 모델 만들어보기 (Dense 노드 수 : 416, Learning Rate : 0.01, batch_size : 100, epochs : 30)
from tensorflow.keras import regularizers

tf.random.set_seed(1442)
initializer = tf.keras.initializers.HeNormal()

model = Sequential()

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01))) # L1 norm regularization))
model.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

model.compile(optimizer=keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model.fit(X_train_scaled, y_train, epochs=2, batch_size=50, validation_data=(X_test_scaled,y_test))

Epoch 1/2
106/106 [==============================] - 2s 9ms/step - loss: 8.4246 - accuracy: 0.7300 - val_loss: 0.5428 - val_accuracy: 0.7344
Epoch 2/2
106/106 [==============================] - 1s 6ms/step - loss: 0.5092 - accuracy: 0.7332 - val_loss: 0.4942 - val_accuracy: 0.8020


In [ ]:
# 테스트셋 사용해서 결과 보기
model.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4942 - accuracy: 0.8020


[0.49415111541748047, 0.8020477890968323]

# 도전과제

- Random Search 하이퍼 매개 변수 튜닝 구현해보세요.
- hyperos 또는 hyperopts를 이용해서 Bayesian Optimiation tuning 수행 [(링크)](https://https://github.com/maxpumperla/hyperas)
- 기존에 진행했던 강의&프로젝트 데이터셋을 하이퍼 파라미터로 조정해보세요. 
- Cifar100을 이용한다면 90% 이상 달성목표! [참고자료](https://paperswithcode.com/sota/image-classification-on-cifar-100)
- MLP 모델을 forward 와 backward(학습까지)를 처음부터 구현할 수 있는가?
- 케라스에서 MLP 모델을 구현하고 교차 검증(CV)을 통해 하이퍼 파라미터를 조정할 수 있는가?



#### 1) Random Search 하이퍼 매개 변수 튜닝 구현해보세요.

In [ ]:
def make_model_rscv(nodes=64, func='relu', rate=0.1, l2=0.01):
  model = Sequential()
  model.add(Dense(nodes,
                  activation='relu',
                  input_dim=X_train_encoded.shape[1]))
  model.add(Dense(256, activation=func))
  model.add(Dropout(rate))
  model.add(Dense(512, activation='relu',
            kernel_regularizer=regularizers.l2(l2)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer = 'adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
tf.random.set_seed(1442)
param_grid = {'nodes': [32, 128, 256, 512],
              'func':['tanh', 'relu', 'sigmoid', 'softmax'],
              'rate':[0, 0.2, 0.5, 0.7],
              'l2':[0.001, 0.005, 0.01, 0.05, 0.1]}

model_rscv = KerasClassifier(build_fn=make_model_rscv, verbose=0)

grid_rscv = RandomizedSearchCV(
    estimator=model_rscv,
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    verbose=3,
    n_jobs=1,
    random_state=12)
grid_result_rscv = grid_rscv.fit(X_train_encoded, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.760, total=   1.5s
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.729, total=   1.5s
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s


[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.737, total=   1.6s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.761, total=   1.4s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.740, total=   1.6s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.762, total=   1.3s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.755, total=   1.4s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.726, total=   1.4s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.763, total=   1.4s
[CV] rate=0.5, nodes=512, l2=0.001, func=si

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.5min finished


In [ ]:
rs = pd.DataFrame(grid_result_rscv.cv_results_).sort_values(by='rank_test_score').head()
rs.T

,1,18,5,16,7
mean_fit_time,1.2708,1.28407,1.23298,1.25501,1.44011
std_fit_time,0.123766,0.123388,0.0152746,0.00997733,0.140968
mean_score_time,0.175653,0.177214,0.183628,0.183925,0.19618
std_score_time,0.00309413,0.00285347,0.00621667,0.00507204,0.00486579
param_rate,0.2,0.2,0.2,0.2,0
param_nodes,32,32,256,256,512
param_l2,0.001,0.005,0.001,0.01,0.001
param_func,relu,softmax,sigmoid,tanh,tanh
params,"{'rate': 0.2, 'nodes': 32, 'l2': 0.001, 'func'...","{'rate': 0.2, 'nodes': 32, 'l2': 0.005, 'func'...","{'rate': 0.2, 'nodes': 256, 'l2': 0.001, 'func...","{'rate': 0.2, 'nodes': 256, 'l2': 0.01, 'func'...","{'rate': 0, 'nodes': 512, 'l2': 0.001, 'func':..."
split0_test_score,0.760533,0.763733,0.759467,0.760533,0.755733


RandomizedSearchCV를 썼을 때의 결과도 GridSearchCV를 사용했을 때와 다르게 나왔다. 하지만 시간 때문에 RandomizedSearchCV의 iter수를 탐색 영역에 비해서 작게 설정했기 때문에 위 결과가 최선의 결과라고 할 수는 없다.

4) MLP 모델을 forward 와 backward(학습까지)를 처음부터 구현할 수 있는가?

In [ ]:
from math import sqrt

In [ ]:
class MLP:
  def __init__(self, input_nodes, hidden_nodes, output_nodes, lr):
    """
    MLP 클래스의 생성자 함수입니다.
    
    Args:
        input_nodes: 입력층의 노드 개수
        hidden_nodes: 은닉층의 노드 개수
        output_nodes: 출력층의 노드 개수
        lr: 학습률
    """
    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes
    self.lr = lr

    self.w_ih = np.random.normal(0,
                                 sqrt(2/(self.input_nodes + self.hidden_nodes)),
                                 (self.input_nodes, self.hidden_nodes))
    self.w_ho = np.random.normal(0,
                                 sqrt(2/(self.hidden_nodes + self.output_nodes)),
                                 (self.hidden_nodes, self.output_nodes))
    

  def sigmoid(self, x):
    """
    활성화 함수로 사용할 sigmoid 함수입니다.
    """
    return 1/(1+np.exp(-x))


  def fit(self, X_train, y_train):
    """
    모델을 학습합니다.

    Args:
        X_train(2차원 numpy.array, float): 학습할 데이터
        y_train(2차원 numpy.array, float): 학습할 데이터의 타겟
    # """
    self.inputs = X_train
    self.targets = y_train
    return self.forward('train')


  def forward(self, filter):
    if filter == 'train':
      self.h = np.dot(self.inputs, self.w_ih)
      self.H = self.sigmoid(self.h)

      self.y = np.dot(self.H, self.w_ho)
      self.Y = self.sigmoid(self.y)

      return self.backward()

    elif filter == 'pred':
      self.h = np.dot(self.inputs, self.w_ih)
      self.H = self.sigmoid(self.h)

      self.y = np.dot(self.H, self.w_ho)
      self.Y = self.sigmoid(self.y)
      return self.Y

  # 가중치 업데이트
  def backward(self):
    self.o_error = -(self.targets-self.Y)
    self.o_delta = self.o_error * (self.Y * (1 - self.Y))

    self.h_error = self.o_delta.dot(self.w_ho.T)
    self.h_delta = self.h_error * (self.y * (1 - self.y))

    self.w_ih = self.w_ih - self.lr*self.inputs.T.dot(self.h_delta)
    self.w_ho = self.w_ho - self.lr*self.H.T.dot(self.o_delta)


  def predict(self, X_test):
    """
    타겟을 예측하는 함수입니다.
    """
    self.inputs = X_test

    result = self.forward('pred')
    return result

In [ ]:
a = np.array([[1, 2], [3, 4], [5, 6]])
b = np.array([[10], [20], [30]])

In [ ]:
mlp = MLP(2, 3, 1, 0.01)

In [ ]:
# 초기 가중치
print(mlp.w_ih)
print(mlp.w_ho)

[[-0.61426513  0.10081131  1.02301425]
 [ 0.26841251 -0.42635879 -0.50733287]]
[[ 0.74888138]
 [ 0.4527325 ]
 [-1.10382178]]


In [ ]:
mlp.fit(a, b)

In [ ]:
# 학습 후 가중치
print(mlp.w_ih)
print(mlp.w_ho)

[[-1.0272452  -0.14885373  1.63173068]
 [-0.23798233 -0.73249726  0.23907326]]
[[ 0.78695969]
 [ 0.47672043]
 [-1.00107793]]


In [ ]:
c = np.array([[10, 12]])

In [ ]:
mlp.predict(c)

array([[0.26873307]])